In [56]:
import pandas as pd
import nltk
import numpy
import sys
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import string
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [16]:
filename = "nursery_rhymes.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
raw_text

'the queen _of_ hearts\n\n\n    the queen of hearts she made some tarts,\n      all on a summer\'s day;\n\n    the knave of hearts he stole those tarts,\n      and took them clean away.\n\n    the king of hearts called for those tarts,\n    and beat the knave full sore.\n\n    the knave of hearts brought back those tarts,\n      and vowed he\'d steal no more.\n\n\n\n\nsaint swithin\'s day\n\n\n    st. swithin\'s day, if thou dost rain,\n    for forty days it will remain;\n    st. swithin\'s day, if thou be fair,\n    for forty days \'t will rain no more.\n\n\n\n\ndance to your daddie\n\n\n      dance to your daddie,\n      my bonnie laddie,\n    dance to your daddie, my bonnie lamb!\n      you shall get a fishie\n      on a little dishie,\n    you shall get a fishie when the boat comes hame!\n\n      dance to your daddie,\n      my bonnie laddie,\n    dance to your daddie, and to your mammie sing!\n      you shall get a coatie,\n      and a pair of breekies,\n    you shall get a coatie

In [17]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
n_chars = len(raw_text)
n_vocab = len(chars)

In [18]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)

In [19]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [20]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [22]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model.fit(X, y, epochs=2, batch_size=128, callbacks=callbacks_list)

Epoch 1/2
804/804 [==============================] - ETA: 0s - loss: 2.5264
Epoch 1: loss improved from inf to 2.52635, saving model to weights-improvement-01-2.5264.hdf5
804/804 [==============================] - 182s 226ms/step - loss: 2.5264
Epoch 2/2
804/804 [==============================] - ETA: 0s - loss: 2.4073
Epoch 2: loss improved from 2.52635 to 2.40729, saving model to weights-improvement-02-2.4073.hdf5
804/804 [==============================] - 178s 221ms/step - loss: 2.4073


In [28]:
filename = "weights-improvement-02-2.4073.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]

" ouse,
      't was built in a wood,
    where an owl at the door
      for sentinel stood.

    she  "
                       ||            ||             ||             ||             ||             ||             ||             ||             ||             ||             ||             ||             ||             ||             ||             ||             ||             ||             ||             ||             ||             ||             ||             ||             ||             ||             ||             ||             ||             ||             ||             ||             ||             ||          

KeyboardInterrupt: 

In [33]:
file = open('nursery_rhymes.txt','r')
with open('nursery_rhymes.txt') as f:
    contents = f.read()
    print(contents)
    data=contents.split('\n')

THE QUEEN _of_ HEARTS


    The Queen of Hearts she made some tarts,
      All on a summer's day;

    The Knave of Hearts he stole those tarts,
      And took them clean away.

    The King of Hearts called for those tarts,
    And beat the Knave full sore.

    The Knave of Hearts brought back those tarts,
      And vowed he'd steal no more.




SAINT SWITHIN'S DAY


    St. Swithin's day, if thou dost rain,
    For forty days it will remain;
    St. Swithin's day, if thou be fair,
    For forty days 't will rain no more.




DANCE TO YOUR DADDIE


      Dance to your daddie,
      My bonnie laddie,
    Dance to your daddie, my bonnie lamb!
      You shall get a fishie
      On a little dishie,
    You shall get a fishie when the boat comes hame!

      Dance to your daddie,
      My bonnie laddie,
    Dance to your daddie, and to your mammie sing!
      You shall get a coatie,
      And a pair of breekies,
    You shall get a coatie when the boat comes in!




THE MAN IN THE MOON




In [36]:
len(data)

5071

In [37]:
data = " ".join(data)

In [38]:
data

'THE QUEEN _of_ HEARTS       The Queen of Hearts she made some tarts,       All on a summer\'s day;      The Knave of Hearts he stole those tarts,       And took them clean away.      The King of Hearts called for those tarts,     And beat the Knave full sore.      The Knave of Hearts brought back those tarts,       And vowed he\'d steal no more.     SAINT SWITHIN\'S DAY       St. Swithin\'s day, if thou dost rain,     For forty days it will remain;     St. Swithin\'s day, if thou be fair,     For forty days \'t will rain no more.     DANCE TO YOUR DADDIE         Dance to your daddie,       My bonnie laddie,     Dance to your daddie, my bonnie lamb!       You shall get a fishie       On a little dishie,     You shall get a fishie when the boat comes hame!        Dance to your daddie,       My bonnie laddie,     Dance to your daddie, and to your mammie sing!       You shall get a coatie,       And a pair of breekies,     You shall get a coatie when the boat comes in!     THE MAN IN THE 

In [42]:
def clean_text(doc):
    tokens = doc.split()
    #Removing punctuation
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    #Removes non alphabet characters
    tokens = [word for word in tokens if word.isalpha()]
    #Makes everything lowercase
    tokens = [word.lower() for word in tokens]
    return tokens

In [43]:
tokens = clean_text(data)
print(tokens[:50])

['the', 'queen', 'of', 'hearts', 'the', 'queen', 'of', 'hearts', 'she', 'made', 'some', 'tarts', 'all', 'on', 'a', 'summers', 'day', 'the', 'knave', 'of', 'hearts', 'he', 'stole', 'those', 'tarts', 'and', 'took', 'them', 'clean', 'away', 'the', 'king', 'of', 'hearts', 'called', 'for', 'those', 'tarts', 'and', 'beat', 'the', 'knave', 'full', 'sore', 'the', 'knave', 'of', 'hearts', 'brought', 'back']


I can now see that the data has been tokenized and now it is easy to read with no random characters included with everything in a unliform character set

In [49]:
print("There are: "+ str(len(set(tokens)))+ " unique words")
print("There are: "+ str(len(tokens))+ " total words")

There are: 2444 unique words
There are: 16786 total words


In [50]:
length = 50 + 1
lines = []

for i in range(length, len(tokens)):
    seq = tokens[i-length:i]
    line = ' '.join(seq)
    lines.append(line)
    if i>200000:
        break
print(len(lines))

16735


In [53]:
lines[3]

'hearts the queen of hearts she made some tarts all on a summers day the knave of hearts he stole those tarts and took them clean away the king of hearts called for those tarts and beat the knave full sore the knave of hearts brought back those tarts and vowed'

Assigning integer values to each word so it can be fed into the model

In [134]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [135]:
sequences=np.array(sequences)
X, y = sequences[:, :-1], sequences[:,-1]

In [136]:
wordDictSize = len(tokenizer.word_index) + 1
y = to_categorical(y, num_classes=wordDictSize)
sequence_length = X.shape[1]

Building model 

In [137]:
print(sequence_length)
model = Sequential()
model.add(Embedding(wordDictSize, 50, input_length = sequence_length))
model.add(LSTM(100, return_sequences= True))
model.add(LSTM(100))
model.add(Dense(100, activation = 'relu'))
model.add(Dense(wordDictSize, activation='softmax'))
model.summary()

50
Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 50, 50)            122250    
                                                                 
 lstm_16 (LSTM)              (None, 50, 100)           60400     
                                                                 
 lstm_17 (LSTM)              (None, 100)               80400     
                                                                 
 dense_16 (Dense)            (None, 100)               10100     
                                                                 
 dense_17 (Dense)            (None, 2445)              246945    
                                                                 
Total params: 520,095
Trainable params: 520,095
Non-trainable params: 0
_________________________________________________________________


In [138]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [139]:
model.fit(X, y, batch_size=256, epochs = 1000)

Epoch 1/1000
66/66 [==============================] - 21s 285ms/step - loss: 6.7920 - accuracy: 0.0457
Epoch 2/1000
66/66 [==============================] - 20s 304ms/step - loss: 6.3182 - accuracy: 0.0546
Epoch 3/1000
66/66 [==============================] - 20s 302ms/step - loss: 6.2979 - accuracy: 0.0546
Epoch 4/1000
66/66 [==============================] - 20s 301ms/step - loss: 6.2329 - accuracy: 0.0546
Epoch 5/1000
66/66 [==============================] - 20s 297ms/step - loss: 6.1273 - accuracy: 0.0538
Epoch 6/1000
66/66 [==============================] - 20s 297ms/step - loss: 6.0714 - accuracy: 0.0578
Epoch 7/1000
66/66 [==============================] - 19s 290ms/step - loss: 5.9901 - accuracy: 0.0645
Epoch 8/1000
66/66 [==============================] - 19s 290ms/step - loss: 5.8822 - accuracy: 0.0688
Epoch 9/1000
66/66 [==============================] - 19s 286ms/step - loss: 5.7942 - accuracy: 0.0733
Epoch 10/1000
66/66 [==============================] - 19s 293ms/step - l

66/66 [==============================] - 19s 286ms/step - loss: 2.7714 - accuracy: 0.3735
Epoch 80/1000
66/66 [==============================] - 19s 280ms/step - loss: 2.7502 - accuracy: 0.3756
Epoch 81/1000
66/66 [==============================] - 19s 282ms/step - loss: 2.7105 - accuracy: 0.3842
Epoch 82/1000
66/66 [==============================] - 19s 282ms/step - loss: 2.6867 - accuracy: 0.3897
Epoch 83/1000
66/66 [==============================] - 18s 280ms/step - loss: 2.6595 - accuracy: 0.3917
Epoch 84/1000
66/66 [==============================] - 19s 283ms/step - loss: 2.6247 - accuracy: 0.4028
Epoch 85/1000
66/66 [==============================] - 18s 278ms/step - loss: 2.6041 - accuracy: 0.4002
Epoch 86/1000
66/66 [==============================] - 19s 281ms/step - loss: 2.5698 - accuracy: 0.4116
Epoch 87/1000
66/66 [==============================] - 19s 282ms/step - loss: 2.5363 - accuracy: 0.4185
Epoch 88/1000
66/66 [==============================] - 18s 278ms/step - loss: 

66/66 [==============================] - 18s 278ms/step - loss: 1.3766 - accuracy: 0.6646
Epoch 158/1000
66/66 [==============================] - 18s 277ms/step - loss: 1.3452 - accuracy: 0.6731
Epoch 159/1000
66/66 [==============================] - 18s 280ms/step - loss: 1.3222 - accuracy: 0.6793
Epoch 160/1000
66/66 [==============================] - 18s 278ms/step - loss: 1.3100 - accuracy: 0.6830
Epoch 161/1000
66/66 [==============================] - 18s 277ms/step - loss: 1.3018 - accuracy: 0.6850
Epoch 162/1000
66/66 [==============================] - 18s 279ms/step - loss: 1.2943 - accuracy: 0.6883
Epoch 163/1000
66/66 [==============================] - 19s 281ms/step - loss: 1.2860 - accuracy: 0.6867
Epoch 164/1000
66/66 [==============================] - 18s 279ms/step - loss: 1.2746 - accuracy: 0.6869
Epoch 165/1000
66/66 [==============================] - 18s 279ms/step - loss: 1.2590 - accuracy: 0.6920
Epoch 166/1000
66/66 [==============================] - 18s 280ms/step

66/66 [==============================] - 19s 287ms/step - loss: 0.7402 - accuracy: 0.8174
Epoch 236/1000
66/66 [==============================] - 19s 286ms/step - loss: 0.7089 - accuracy: 0.8255
Epoch 237/1000
66/66 [==============================] - 19s 287ms/step - loss: 0.6974 - accuracy: 0.8325
Epoch 238/1000
66/66 [==============================] - 19s 287ms/step - loss: 0.6846 - accuracy: 0.8366
Epoch 239/1000
66/66 [==============================] - 19s 286ms/step - loss: 0.6739 - accuracy: 0.8390
Epoch 240/1000
66/66 [==============================] - 19s 287ms/step - loss: 0.6709 - accuracy: 0.8386
Epoch 241/1000
66/66 [==============================] - 19s 286ms/step - loss: 0.6598 - accuracy: 0.8430
Epoch 242/1000
66/66 [==============================] - 19s 286ms/step - loss: 0.6588 - accuracy: 0.8416
Epoch 243/1000
66/66 [==============================] - 19s 287ms/step - loss: 0.6628 - accuracy: 0.8387
Epoch 244/1000
66/66 [==============================] - 19s 286ms/step

66/66 [==============================] - 19s 291ms/step - loss: 0.2968 - accuracy: 0.9385
Epoch 314/1000
66/66 [==============================] - 19s 291ms/step - loss: 0.2897 - accuracy: 0.9404
Epoch 315/1000
66/66 [==============================] - 19s 291ms/step - loss: 0.2813 - accuracy: 0.9430
Epoch 316/1000
66/66 [==============================] - 19s 292ms/step - loss: 0.2808 - accuracy: 0.9428
Epoch 317/1000
66/66 [==============================] - 19s 291ms/step - loss: 0.2835 - accuracy: 0.9424
Epoch 318/1000
66/66 [==============================] - 19s 292ms/step - loss: 0.2831 - accuracy: 0.9412
Epoch 319/1000
66/66 [==============================] - 19s 292ms/step - loss: 0.2917 - accuracy: 0.9388
Epoch 320/1000
66/66 [==============================] - 19s 293ms/step - loss: 0.2992 - accuracy: 0.9362
Epoch 321/1000
66/66 [==============================] - 19s 293ms/step - loss: 0.2960 - accuracy: 0.9341
Epoch 322/1000
66/66 [==============================] - 19s 292ms/step

...

66/66 [==============================] - 22s 333ms/step - loss: 0.2815 - accuracy: 0.9178
Epoch 626/1000
66/66 [==============================] - 22s 328ms/step - loss: 0.1491 - accuracy: 0.9567
Epoch 627/1000
66/66 [==============================] - 22s 328ms/step - loss: 0.0832 - accuracy: 0.9803
Epoch 628/1000
66/66 [==============================] - 22s 328ms/step - loss: 0.0396 - accuracy: 0.9926
Epoch 629/1000
66/66 [==============================] - 22s 328ms/step - loss: 0.0203 - accuracy: 0.9978
Epoch 630/1000
66/66 [==============================] - 22s 331ms/step - loss: 0.0143 - accuracy: 0.9983
Epoch 631/1000
66/66 [==============================] - 22s 329ms/step - loss: 0.0127 - accuracy: 0.9984
Epoch 632/1000
66/66 [==============================] - 22s 329ms/step - loss: 0.0120 - accuracy: 0.9983
Epoch 633/1000
66/66 [==============================] - 22s 327ms/step - loss: 0.0117 - accuracy: 0.9981
Epoch 634/1000
66/66 [==============================] - 22s 329ms/step

66/66 [==============================] - 22s 335ms/step - loss: 0.0094 - accuracy: 0.9989
Epoch 704/1000
66/66 [==============================] - 22s 331ms/step - loss: 0.0091 - accuracy: 0.9985
Epoch 705/1000
66/66 [==============================] - 22s 332ms/step - loss: 0.0086 - accuracy: 0.9989
Epoch 706/1000
66/66 [==============================] - 22s 332ms/step - loss: 0.0079 - accuracy: 0.9988
Epoch 707/1000
66/66 [==============================] - 22s 333ms/step - loss: 0.0080 - accuracy: 0.9987
Epoch 708/1000
66/66 [==============================] - 22s 330ms/step - loss: 0.0080 - accuracy: 0.9986
Epoch 709/1000
66/66 [==============================] - 22s 332ms/step - loss: 0.0080 - accuracy: 0.9987
Epoch 710/1000
66/66 [==============================] - 22s 332ms/step - loss: 0.0077 - accuracy: 0.9987
Epoch 711/1000
66/66 [==============================] - 22s 336ms/step - loss: 0.0074 - accuracy: 0.9987
Epoch 712/1000
66/66 [==============================] - 22s 333ms/step

66/66 [==============================] - 22s 331ms/step - loss: 0.0111 - accuracy: 0.9983
Epoch 782/1000
66/66 [==============================] - 22s 333ms/step - loss: 0.0093 - accuracy: 0.9987
Epoch 783/1000
66/66 [==============================] - 22s 331ms/step - loss: 0.0081 - accuracy: 0.9986
Epoch 784/1000
66/66 [==============================] - 22s 334ms/step - loss: 0.0074 - accuracy: 0.9987
Epoch 785/1000
66/66 [==============================] - 22s 334ms/step - loss: 0.0073 - accuracy: 0.9989
Epoch 786/1000
66/66 [==============================] - 22s 339ms/step - loss: 0.0072 - accuracy: 0.9988
Epoch 787/1000
66/66 [==============================] - 22s 339ms/step - loss: 0.0070 - accuracy: 0.9987
Epoch 788/1000
66/66 [==============================] - 22s 337ms/step - loss: 0.0077 - accuracy: 0.9987
Epoch 789/1000
66/66 [==============================] - 22s 335ms/step - loss: 0.0063 - accuracy: 0.9990
Epoch 790/1000
66/66 [==============================] - 22s 335ms/step

66/66 [==============================] - 22s 339ms/step - loss: 0.5742 - accuracy: 0.8524
Epoch 860/1000
66/66 [==============================] - 22s 336ms/step - loss: 0.2659 - accuracy: 0.9229
Epoch 861/1000
66/66 [==============================] - 22s 339ms/step - loss: 0.0881 - accuracy: 0.9758
Epoch 862/1000
66/66 [==============================] - 22s 338ms/step - loss: 0.0276 - accuracy: 0.9944
Epoch 863/1000
66/66 [==============================] - 22s 339ms/step - loss: 0.0153 - accuracy: 0.9980
Epoch 864/1000
66/66 [==============================] - 22s 339ms/step - loss: 0.0084 - accuracy: 0.9989
Epoch 865/1000
66/66 [==============================] - 22s 338ms/step - loss: 0.0065 - accuracy: 0.9990
Epoch 866/1000
66/66 [==============================] - 22s 337ms/step - loss: 0.0063 - accuracy: 0.9991
Epoch 867/1000
66/66 [==============================] - 22s 339ms/step - loss: 0.0058 - accuracy: 0.9991
Epoch 868/1000
66/66 [==============================] - 22s 337ms/step

66/66 [==============================] - 22s 340ms/step - loss: 0.0040 - accuracy: 0.9990
Epoch 938/1000
66/66 [==============================] - 22s 340ms/step - loss: 0.0032 - accuracy: 0.9993
Epoch 939/1000
66/66 [==============================] - 22s 340ms/step - loss: 0.0033 - accuracy: 0.9993
Epoch 940/1000
66/66 [==============================] - 23s 341ms/step - loss: 0.0032 - accuracy: 0.9992
Epoch 941/1000
66/66 [==============================] - 22s 339ms/step - loss: 0.0038 - accuracy: 0.9993
Epoch 942/1000
66/66 [==============================] - 22s 340ms/step - loss: 0.0042 - accuracy: 0.9992
Epoch 943/1000
66/66 [==============================] - 23s 341ms/step - loss: 0.0034 - accuracy: 0.9993
Epoch 944/1000
66/66 [==============================] - 22s 339ms/step - loss: 0.0034 - accuracy: 0.9993
Epoch 945/1000
66/66 [==============================] - 22s 339ms/step - loss: 0.0032 - accuracy: 0.9992
Epoch 946/1000
66/66 [==============================] - 22s 340ms/step

In [140]:
seed=(lines[123])


In [141]:
def create_rhyme(model, tokenizer, text_seq_length, seed, num_words):
    rhyme=[]
    for _ in range(num_words):
        encoded = tokenizer.texts_to_sequences([seed])[0]
        encoded=pad_sequences([encoded], maxlen = text_seq_length, truncating='pre')
        
        predict_y=model.predict(encoded) 
        y_pred=np.argmax(predict_y,axis=1)
        pred_word =''
        for word, index in tokenizer.word_index.items():
            print(y_pred)
            if (index == y_pred):
                pred_word = word
                break
        seed = seed + " " + pred_word
        rhyme.append(pred_word)
    return ' '.join(rhyme)

In [142]:

create_rhyme(model, tokenizer, sequence_length, seed, 100)

[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[44]
[2]
[2]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708]
[708

[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]
[1373]

[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564]
[564

[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]
[1374]

[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]
[1375]

...

[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]
[1378]

[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359]
[359

[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[95]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[112]
[1

[39]
[39]
[39]
[39]
[39]
[39]
[39]
[39]
[39]
[39]
[39]
[39]
[39]
[39]
[39]
[39]
[39]
[39]
[39]
[39]
[39]
[39]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[91]
[3]
[3]
[3]
[8]
[8]
[8]
[8]
[8]
[8]
[8]
[8]


'down and asked the way to norwich he went by the south and burnt his mouth with eating cold pease porridge simple simon simple simon met a pieman going to the fair says simple simon to the pieman let me taste your ware says the pieman unto simon first give me a penny says simple simon to the pieman i have not got any he went to catch a dickybird and thought he could not fail because he had got a little salt to put upon his tail he went to ride a spotted cow that had got a little'